In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lightning import pytorch as pl
from sklearn.model_selection import train_test_split
import torch
torch.set_float32_matmul_precision('medium')
from chemprop import data, featurizers, models, nn
pl.seed_everything(1)
from utils import *
# from data_utils import data_prep, prep_data, load_data, split_data, preprocess_data, create_data_loaders, create_mpnn_model
%load_ext autoreload
%autoreload 2

Seed set to 1
Seed set to 1


In [2]:
input_path = r'C:\Users\rhys-\OneDrive\Documents\Github\chemprop\notebooks\big_data\data\ir_dataset_minmax_scaled.parquet' # path to your data .csv file
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = "smiles" # name of the column containing SMILES strings
target_columns = np.arange(400,4002,2).astype(str) # list of names of the columns containing targets

df_input = pd.read_parquet(input_path)

In [3]:
nist_test_path = r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\data\nist\nist_test.parquet"

nist_test_df = pd.read_parquet(nist_test_path)

test_datapoints = get_mol_datapoints(nist_test_df, smiles_column, target_columns)

test_dset = data.MoleculeDataset(test_datapoints, featurizers.SimpleMoleculeMolGraphFeaturizer())

test_loader = data.MolGraphDataLoader(test_dset, num_workers=num_workers)

In [4]:
def create_rndm_loader(test_data):
    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()


    test_dset = data.MoleculeDataset(test_data, featurizer)

    return data.MolGraphDataLoader(test_dset, num_workers=0)

rndm_test_loader = create_rndm_loader(test_1[:700])

NameError: name 'test_1' is not defined

In [5]:
datapoints = get_mol_datapoints(df_input, smiles_column, target_columns)

train_idx, val_idx, test_idx = data.splitting.split_data(
    datapoints,
    num_folds=8,
    split='cv',
)

train_1 = [datapoints[i] for i in train_idx[0]]
val_1 = [datapoints[i] for i in val_idx[0]]
test_1 = [datapoints[i] for i in test_idx[0]]

train_2 = [datapoints[i] for i in train_idx[1]]
val_2 = [datapoints[i] for i in val_idx[1]]
test_2 = [datapoints[i] for i in test_idx[1]]

train_3 = [datapoints[i] for i in train_idx[2]]
val_3 = [datapoints[i] for i in val_idx[2]]
test_3 = [datapoints[i] for i in test_idx[2]]

train_4 = [datapoints[i] for i in train_idx[3]]
val_4 = [datapoints[i] for i in val_idx[3]]
test_4 = [datapoints[i] for i in test_idx[3]]

train_5 = [datapoints[i] for i in train_idx[4]]
val_5 = [datapoints[i] for i in val_idx[4]]
test_5 = [datapoints[i] for i in test_idx[4]]

train_6 = [datapoints[i] for i in train_idx[5]]
val_6 = [datapoints[i] for i in val_idx[5]]
test_6= [datapoints[i] for i in test_idx[5]]

train_7 = [datapoints[i] for i in train_idx[6]]
val_7 = [datapoints[i] for i in val_idx[6]]
test_7 = [datapoints[i] for i in test_idx[6]]

train_8 = [datapoints[i] for i in train_idx[7]]
val_8 = [datapoints[i] for i in val_idx[7]]
test_8 = [datapoints[i] for i in test_idx[7]]

def create_data_loaders(train_data, val_data, test_data, num_workers=0):
    featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

    train_dset = data.MoleculeDataset(train_data, featurizer)
    val_dset = data.MoleculeDataset(val_data, featurizer)
    test_dset = data.MoleculeDataset(test_data, featurizer)

    train_loader = data.MolGraphDataLoader(train_dset, num_workers=num_workers)
    val_loader = data.MolGraphDataLoader(val_dset, num_workers=num_workers, shuffle=False)
    test_loader = data.MolGraphDataLoader(test_dset, num_workers=num_workers, shuffle=False)

    return train_loader, val_loader, test_loader

train_loader_1, val_loader_1, test_loader_1 = create_data_loaders(train_1, val_1, test_1, num_workers)
train_loader_2, val_loader_2, test_loader_2 = create_data_loaders(train_2, val_2, test_2, num_workers)
train_loader_3, val_loader_3, test_loader_3 = create_data_loaders(train_3, val_3, test_3, num_workers)
train_loader_4, val_loader_4, test_loader_4 = create_data_loaders(train_4, val_4, test_4, num_workers)
train_loader_5, val_loader_5, test_loader_5 = create_data_loaders(train_5, val_5, test_5, num_workers)
train_loader_6, val_loader_6, test_loader_6 = create_data_loaders(train_6, val_6, test_6, num_workers)
train_loader_7, val_loader_7, test_loader_7 = create_data_loaders(train_7, val_7, test_7, num_workers)
train_loader_8, val_loader_8, test_loader_8 = create_data_loaders(train_8, val_8, test_8, num_workers)

In [6]:
def create_model(metric: str, agg: str = 'mean'):
    mp = nn.BondMessagePassing(
    depth = 6,
    d_h = 2200,
    dropout=0.05,
    # activation='SILU'
)
    if agg == 'mean':
        agg = nn.MeanAggregation(

        )

    elif agg == 'attentive':
        agg = nn.AttentiveAggregation(
            output_size=2200,
        )

    elif agg == 'sum': 
        agg = nn.SumAggregation(
        )
    else:
        raise ValueError(f'Invalid aggregation: {agg}')

    # agg = nn.AttentiveAggregation(
    #     output_size=2200,
    # )

    ffn = nn.RegressionFFN(
        input_dim=2200,
        n_layers=6,
        hidden_dim=2200,
        dropout=0.05,
        # activation='SILU',
        # loc=scaler.mean_, # pass in the mean of the training targets
        # scale=scaler.scale_,
        n_tasks=1801 # pass in the scale of the training targets
    )
    batch_norm=False
    if metric == 'rmse':
        metric_list = [nn.RMSEMetric()]
    elif metric == 'sid':
        metric_list = [nn.SIDMetric(), nn.RMSEMetric()]

    else:
        raise ValueError(f'Invalid metric: {metric}')
    return models.MPNN(mp, agg, ffn, batch_norm, metrics=metric_list)

mpnn1 = create_model('rmse', 'attentive')
mpnn2 = create_model('rmse', 'attentive')
mpnn3 = create_model('rmse', 'attentive')
mpnn4 = create_model('rmse', 'attentive')
mpnn5 = create_model('rmse', 'attentive')
mpnn6 = create_model('rmse', 'attentive')
mpnn7 = create_model('rmse', 'attentive')
mpnn8 = create_model('rmse', 'attentive')


In [8]:
trainer_1 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_2 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
)

trainer_3 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_4 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
)

trainer_5 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_6 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
)

trainer_7 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
)

trainer_8 = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=12, # number of epochs to train for
)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\logger_connector\logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: Tru

In [9]:
trainer_1.fit(mpnn1, train_loader_1, val_loader_1)
trainer_2.fit(mpnn2, train_loader_2, val_loader_2)
trainer_3.fit(mpnn3, train_loader_3, val_loader_3)
trainer_4.fit(mpnn4, train_loader_4, val_loader_4)
trainer_5.fit(mpnn5, train_loader_5, val_loader_5)
trainer_6.fit(mpnn6, train_loader_6, val_loader_6)
trainer_7.fit(mpnn7, train_loader_7, val_loader_7)
trainer_8.fit(mpnn8, train_loader_8, val_loader_8)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.

  | Name            | Type                 | Params
---------------------------------------------------------
0 | message_passing | BondMessagePassing   | 10.3 M
1 | agg             | AttentiveAggregation | 2.2 K 
2 | bn              | Identity             | 0     
3 | predictor       | RegressionFFN        | 33.0 M
  | other params    | n/a                  | 1.8 K 
---------------------------------------------------------
43.3 M    Trainable params
1.8 K     Non-trainable params
43.3 M    Total params
173.278   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:  26%|██▌       | 500/1948 [02:30<07:16,  3.32it/s, v_num=100, train/loss=0.0057]  

c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type                 | Params
---------------------------------------------------------
0 | message_passing | BondMessagePassing   | 10.3 M
1 | agg             | AttentiveAggregation | 2.2 K 
2 | bn              | Identity             | 0     
3 | predictor       | RegressionFFN        | 33.0 M
  | other params    | n/a                  | 1.8 K 
---------------------------------------------------------
43.3 M    Trainable params
1.8 K     Non-trainable params
43.3 M    Total params
173.278   Total estimated model params size (MB)


Epoch 0:  29%|██▉       | 565/1948 [00:33<01:21, 16.99it/s, v_num=101, train/loss=0.00521]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name            | Type                 | Params
---------------------------------------------------------
0 | message_passing | BondMessagePassing   | 10.3 M
1 | agg             | AttentiveAggregation | 2.2 K 
2 | bn              | Identity             | 0     
3 | predictor       | RegressionFFN        | 33.0 M
  | other params    | n/a                  | 1.8 K 
---------------------------------------------------------
43.3 M    Trainable params
1.8 K     Non-trainable params
43.3 M    Total params
173.278   Total estimated model params size (MB)


Epoch 0:   4%|▍         | 83/1948 [00:11<04:20,  7.16it/s, v_num=102, train/loss=0.00656]

In [ ]:
from chemprop import models 

trainer_1.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn1.ckpt')
trainer_2.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn2.ckpt')
trainer_3.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn3.ckpt')
trainer_4.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn4.ckpt')
trainer_5.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn5.ckpt')
trainer_6.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn6.ckpt')
trainer_7.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn7.ckpt')
trainer_8.save_checkpoint(r'C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn8.ckpt')

checkpoint = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn1.ckpt")
checkpoint2 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn2.ckpt")
checkpoint3 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn3.ckpt")
checkpoint4 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn4.ckpt")
checkpoint5 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn5.ckpt")
checkpoint6 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn6.ckpt")
checkpoint7 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn7.ckpt")
checkpoint8 = torch.load(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\mpnn8.ckpt")

# Assuming 'output_size' needs to be added or corrected
# You need to know the correct value for 'output_size'
checkpoint['hyper_parameters']['agg']['output_size'] = 2200
checkpoint2['hyper_parameters']['agg']['output_size'] = 2200
checkpoint3['hyper_parameters']['agg']['output_size'] = 2200
checkpoint4['hyper_parameters']['agg']['output_size'] = 2200
checkpoint5['hyper_parameters']['agg']['output_size'] = 2200
checkpoint6['hyper_parameters']['agg']['output_size'] = 2200
checkpoint7['hyper_parameters']['agg']['output_size'] = 2200
checkpoint8['hyper_parameters']['agg']['output_size'] = 2200

torch.save(checkpoint, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_1.ckpt")
torch.save(checkpoint2, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_2.ckpt")
torch.save(checkpoint3, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_3.ckpt")
torch.save(checkpoint4, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_4.ckpt")
torch.save(checkpoint5, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_5.ckpt")
torch.save(checkpoint6, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_6.ckpt")
torch.save(checkpoint7, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_7.ckpt")
torch.save(checkpoint8, r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_8.ckpt")



In [ ]:
loaded_model = models.MPNN.load_from_checkpoint(r"C:\Users\rhys-\OneDrive\Documents\Github\thesis_results\models\modified_mpnn_1.ckpt")


In [ ]:
trainer_loaded = pl.Trainer(
    precision='bf16-mixed',
    logger=True,
    enable_checkpointing=False, # Use `True` if you want to save model checkpoints. The checkpoints will be saved in the `checkpoints` folder.
    enable_progress_bar=True,
    accelerator="auto",
    devices=1,
    max_epochs=6, # number of epochs to train for
    # callbacks=[metric_tracker, early_stop_callback]
)

trainer_loaded.test(loaded_model, test_loader_1)

Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\rhys-\miniconda3\envs\cp3\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 325/325 [00:07<00:00, 40.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test/rmse          0.057879265397787094
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test/rmse': 0.057879265397787094}]